In [3]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# Python optimisation variables
learning_rate = 0.0001
epochs = 1
batch_size = 50

# declare the training data placeholders
# input x - for 28 x 28 pixels = 784 - this is the flattened image data that is drawn from 
# mnist.train.nextbatch()
x = tf.placeholder(tf.float32, [None, 784])
# dynamically reshape the input
x_shaped = tf.reshape(x, [-1, 28, 28, 1])
# now declare the output data placeholder - 10 digits
y = tf.placeholder(tf.float32, [None, 10])

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
from tensorflow.python.training.adam import AdamOptimizer
#tf.train.AdamOptimizer

In [6]:
class AdamOptimizer_Bing(AdamOptimizer):
    def __init__(self, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8,
               use_locking=False, name="Adam"):

        super(AdamOptimizer, self).__init__(use_locking, name)
        self._lr = learning_rate
        self._beta1 = beta1
        self._beta2 = beta2
        self._epsilon = epsilon

        # Tensor versions of the constructor arguments, created in _prepare().
        self._lr_t = None
        self._beta1_t = None
        self._beta2_t = None
        self._epsilon_t = None

        # Created in SparseApply if needed.
        self._updated_lr = None
    
    def minimize(self, loss, global_step=None, var_list=None,
               gate_gradients=1, aggregation_method=None,
               colocate_gradients_with_ops=False, name=None,
               grad_loss=None):
        """
        The same as function minimize, but return the result of compute_gradients
        Created by: Big Bing in 7/28 
        Purpose: To realize parallel computing(communicate gradient)
        """
        grads_and_vars = self.compute_gradients(
            loss, var_list=var_list, gate_gradients=gate_gradients,
            aggregation_method=aggregation_method,
            colocate_gradients_with_ops=colocate_gradients_with_ops,
            grad_loss=grad_loss)[-8:]

        vars_with_grad = [v for g, v in grads_and_vars if g is not None]
        if not vars_with_grad:
          raise ValueError(
              "No gradients provided for any variable, check your graph for ops"
              " that do not support gradients, between variables %s and loss %s." %
              ([str(v) for _, v in grads_and_vars], loss))
        #self.apply_gradients(grads_and_vars, global_step=global_step, name=name)

        return self.apply_gradients(grads_and_vars, global_step=global_step, name=name),grads_and_vars

In [7]:
def create_new_conv_layer(input_data, num_input_channels, num_filters, filter_shape, pool_shape, name):
    # setup the filter input shape for tf.nn.conv_2d
    conv_filt_shape = [filter_shape[0], filter_shape[1], num_input_channels,
                      num_filters]

    # initialise weights and bias for the filter
    weights = tf.Variable(tf.truncated_normal(conv_filt_shape, stddev=0.03),
                                      name=name+'_W')
    bias = tf.Variable(tf.truncated_normal([num_filters]), name=name+'_b')

    # setup the convolutional layer operation
    out_layer = tf.nn.conv2d(input_data, weights, [1, 1, 1, 1], padding='SAME')

    # add the bias
    out_layer += bias

    # apply a ReLU non-linear activation
    out_layer = tf.nn.relu(out_layer)

    # now perform max pooling
    ksize = [1, pool_shape[0], pool_shape[1], 1]
    strides = [1, 2, 2, 1]
    out_layer = tf.nn.max_pool(out_layer, ksize=ksize, strides=strides, 
                               padding='SAME')

    return out_layer

In [8]:
# create some convolutional layers
layer1 = create_new_conv_layer(x_shaped, 1, 32, [5, 5], [2, 2], name='layer1')
layer2 = create_new_conv_layer(layer1, 32, 64, [5, 5], [2, 2], name='layer2')

In [9]:
flattened = tf.reshape(layer2, [-1, 7 * 7 * 64])

In [10]:
# setup some weights and bias values for this layer, then activate with ReLU
wd1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1000], stddev=0.03), name='wd1')
bd1 = tf.Variable(tf.truncated_normal([1000], stddev=0.01), name='bd1')
dense_layer1 = tf.matmul(flattened, wd1) + bd1
dense_layer1 = tf.nn.relu(dense_layer1)

In [11]:
# another layer with softmax activations
wd2 = tf.Variable(tf.truncated_normal([1000, 10], stddev=0.03), name='wd2')
bd2 = tf.Variable(tf.truncated_normal([10], stddev=0.01), name='bd2')
dense_layer2 = tf.matmul(dense_layer1, wd2) + bd2
y_ = tf.nn.softmax(dense_layer2)

In [12]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=dense_layer2, labels=y))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [13]:
# add an optimiser
optimizer_gradient = AdamOptimizer_Bing(learning_rate=learning_rate).minimize(cross_entropy)
optimizer = AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

In [14]:
# setup the initialisation operator
init_op = tf.global_variables_initializer()
grad = []
with tf.Session() as sess:
    # initialise the variables
    sess.run(init_op)
    total_batch = int(len(mnist.train.labels))
    count = 0
    for epoch in range(1):
        batch_x, batch_y = mnist.train.next_batch(batch_size=total_batch)
        g,c = sess.run([optimizer_gradient,cross_entropy], feed_dict={x: batch_x, y: batch_y})
        
    print("\nTraining complete!")



Training complete!


In [17]:
g = np.array(g[1])

In [31]:
b = g[:,0]

In [34]:
b[4].shape

(3136, 1000)

In [20]:
a = batch_parameter_collector(g)

In [22]:
a[0].shape

(800,)

In [2]:
c = []
for i in range(8):
    a = g[i][1]
    c.append(a)

NameError: name 'g' is not defined

In [19]:
def batch_parameter_collector(paramter):
    '''
    Collect the gradient of each batch
    Input: the return value of optimizer.compute_gradients (grad_var list)
    Output: the sum of gradients within one epoch as a vector
    
    '''
    paramter = np.array(paramter)
    #shape_list record parameter shapes of each layer
    parameter_vector = []
    for i in range(paramter.shape[0]):
        parameter_temp = paramter[i,1].flatten()
        parameter_vector.append(parameter_temp)
        
    return np.array(parameter_vector)

In [41]:
c = c.flatten()

In [46]:
c[0][0]

array([[[ 0.03369243, -0.03081585,  0.002443  , -0.03168654,
          0.01637692,  0.01739312, -0.00290238,  0.02563729,
         -0.03621509, -0.02130874, -0.00868434,  0.04764776,
          0.01160116, -0.0274205 , -0.0165049 ,  0.01610075,
         -0.02719073,  0.00178517,  0.00649393,  0.04148226,
         -0.03355931,  0.03307768, -0.011563  , -0.02181776,
          0.03210377, -0.00106341, -0.00159062, -0.00045102,
         -0.01758818,  0.00028138,  0.00488585, -0.0129739 ]],

       [[-0.05502194,  0.01712925, -0.00403057, -0.02017778,
         -0.03627938, -0.02287476, -0.02205104, -0.01227884,
          0.00573386, -0.0076336 ,  0.00929266, -0.03580606,
          0.0045444 ,  0.05044917,  0.01038548,  0.00268515,
         -0.02750078,  0.01958369,  0.00554364, -0.03375189,
         -0.00480494,  0.02911807, -0.0040464 , -0.00561467,
         -0.05437297, -0.01555247, -0.01706062,  0.01919681,
         -0.00066995,  0.01664072,  0.03745938, -0.01683863]],

       [[-0.001818

In [33]:
b = b.flatten()

In [ ]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


# setup the initialisation operator
init_op = tf.global_variables_initializer()
grad = []
with tf.Session() as sess:
    # initialise the variables
    sess.run(init_op)
    total_batch = int(len(mnist.train.labels) / batch_size)
    count = 0
    for epoch in range(1):
        avg_cost = 0
        if count == epochs - 1:
            for i in range(total_batch):
                batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
                g,c = sess.run([optimizer_gradient,cross_entropy], feed_dict={x: batch_x, y: batch_y})
                gradient_temp = batch_gradient_collector(g[1])
                grad.append(gradient_temp)
                avg_cost += c / total_batch
            shape_list = parameter_shape(g[1])
            #Sum up gradients from each batch
            gradient = np.array(grad).sum(axis = 0)
        
        else:
            print('SB')
            for i in range(total_batch):
                batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
                _,c = sess.run([optimizer,cross_entropy], feed_dict={x: batch_x, y: batch_y})
                avg_cost += c / total_batch
            
            
        count += 1   
        test_acc = sess.run(accuracy, 
                       feed_dict={x: mnist.test.images, y: mnist.test.labels})
        print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost), "test accuracy: {:.3f}".format(test_acc))

    print("\nTraining complete!")
    print(sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels}))

In [14]:
def batch_gradient_collector(gradient):
    '''
    Collect the gradient of each batch
    Input: the return value of optimizer.compute_gradients (many matrices)
    Output: the sum of gradients within one epoch as a vector
    
    '''
    gradient = np.array(gradient)
    #shape_list record parameter shapes of each layer
    gradient_vector = []
    for i in range(gradient.shape[0]):
        grad_temp = gradient[i,0].flatten()
        gradient_vector.append(grad_temp)
        
    return np.array(gradient_vector)
        

In [15]:
def parameter_shape(gradient):
    '''
    Record the parameters original shape
    Input: the return value of optimizer.compute_gradients (many matrices)
    Output: A list of shape
    
    '''
    gradient = np.array(gradient)
    #shape_list record parameter shapes of each layer
    shape_list = []
    for i in range(gradient.shape[0]):
        grad_temp = gradient[i,0].flatten()
        shape_list.append(['Layer_'+str(i+1), gradient[i,0].shape, grad_temp.shape[0]])
        
    return shape_list

In [1]:
g

NameError: name 'g' is not defined